# Imports

In [ ]:
# automaticlly updates imported methods when they change
%load_ext autoreload
%autoreload 2

In [ ]:
# %%
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.parent))
from IPython import get_ipython
from plots import plot_meas_vs_bb, plot_surface_fit
from tools import radius, fit_surface
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
import threading as th
from functools import partial
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

---

# Load measurements

## Set path to the data

In [ ]:
path = Path().cwd()
while 'Multi' not in path.name:
    path = path.parent
PATH_TO_LOGS = path / 'data' / 'meas'

## Load

---

In [ ]:
def _loader(path):
    return str(path).split("\\")[-1], np.load(path)

paths = list(PATH_TO_LOGS.glob('*.npy'))
with Pool(cpu_count()) as pool:
        loaded_data = list(tqdm(pool.imap(_loader, paths), desc='Loading data', total=len(paths)))
loaded_data = sorted(loaded_data, key=lambda x: x[0])
list_wavelength = list(pd.read_csv(list(PATH_TO_LOGS.glob('*.csv'))[0]).iloc[:, 1])

## Plot measurements

# Temperature estimation based on grey-levels:
The relationship between the Black-Bodies radiance and the obtained grey-levels can be captured by the formula:
$$G_{i,j} = a_{i,j} \cdot P(\lambda_c, T)$$
Where:

* $G_{i,j}$ is the grey level of the i,jth pixel.
* $P(\lambda_c, T)$ is the black-body's radiance at the temperature $T$, after being filtered by a narrow band-pass with a central frequency $\lambda_c$. This term can be analytically solved for:
    $$P(\lambda_c, T) = \int_{\lambda_c-\frac{bw}{2}}^{\lambda_c+\frac{bw}{2}} B(\lambda, T) d\lambda$$ 
    where $B(\lambda, T) = \frac{2h/c^2\lambda^3}{exp(h/KT\lambda)-1}$, and $bw$ is the narrow band-pass band-width.
* $a_{i,j} are the scaling coefficients that translate the filtered radiance into grey-levels

Hence, to estimate the grey-levels, we should:
1. Estimate the black-body's radiance for the different temperature and filters.
2. Solve for the scaling coefficients.
3. Solve for the temperature (possibly be a LS solution to the primary equation above) 

## Estimate the black-body radiance coefficients per temperature and central frequency:
To estimate the Black-bodies radiance, we need to calculate the integral presented above:
$$P(\lambda_c, T) = \int_{\lambda_c-\frac{bw}{2}}^{\lambda_c+\frac{bw}{2}} B(\lambda, T) d\lambda$$ 

However, as our data is acquired using real-world filters (and not ideal ones), the non-optimal transmitance needs to be acounted for. Threfore, the actual radiance received by the sensor is:
$$P(\lambda_c, T) = \int_{\lambda_c-\frac{bw}{2}}^{\lambda_c+\frac{bw}{2}} B(\lambda, T)\cdot F(\lambda) d\lambda$$ 
where $F(\lambda)$ is the frequency response of our narrow band-pass IR filter.

The function ***calcRxPower*** acounts for these non-optimalities, and calculates the actual received radiance according to the temperature and central frequency. To do so, it looks for the frequency response of the relevant filter in a static xlsx table containing Thorlab's characterizations for each filter.

Along is an example of how the function works at $T=32[C]$, $\lambda_c=9000[nm]$: 


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams["font.size"] = 14
matplotlib.rcParams["figure.figsize"] = (19, 10)
from tools import calcRxPower
rx_power = calcRxPower(32, 9000, debug=True)


With that functionality in hand, let's now calculate the received radiance for each set point of temperature and filter:

In [ ]:
# %% 
import pandas as pd
c_wls = measurements.keys()  # list of central wavelengths
temperatures = measurements[list(c_wls)[0]].keys()
plank_coeefs = pd.DataFrame(index=temperatures, columns=c_wls)
for c_wl in c_wls:
    for temp in temperatures:
        plank_coeefs.loc[temp, c_wl] = calcRxPower(temp, c_wl)

plt.figure()
plank_coeefs.plot(style='-o')
plt.title("$P(\lambda, T)$ Vs Temperature")
plt.ylabel("$P(\lambda, T)$")
plt.xlabel("Black-Body Temperature[C]")
plt.grid()


## Solve for the per-pixel scaling factors ($a_{i,j}$)    

### 1st approach: normalize grey levels by $P(\lambda, T)$ and average over all samples

In [ ]:
# Averaging an Normalizing:
chunk_shape = image.shape
all_frames_aligned = np.zeros(shape=[len(c_wls)*len(temperatures), *chunk_shape[1:]])
for i, c_wl in enumerate(c_wls):
    for j, temp in enumerate(temperatures):
        all_frames_aligned[i*len(temperatures)+j, ...] = measurements[c_wl][temp].mean(axis=0) / plank_coeefs.loc[temp, c_wl]

a_ij = all_frames_aligned.mean(axis=0)


In [ ]:
plt.figure()
plt.imshow(a_ij, cmap="gray")
plt.title("scaling factors $a_{i,j}$")

#### Validate Results:
We will validate by normalizing an average frame at a given temperature and central frequency by the scaling coefficients. We expect to obtain a roughly constant result, which reflects the spectral radiance at this temperature and central frequency:

$$G_{i,j} = a_{i,j} \cdot P(\lambda_c, T) \Rightarrow \frac{G_{i,j}}{a_{i,j}} = P(\lambda_c, T)$$

In [ ]:
arbitrary_frame = measurements[c_wl][temp].mean(axis=0)
radiometric_range = arbitrary_frame.ptp()
frame_center = np.mean([arbitrary_frame.min(), arbitrary_frame.max()])
P_hat = arbitrary_frame / a_ij
P_hat_center = np.mean([P_hat.min(), P_hat.max()])
fig, ax = plt.subplots(2, 2)
ax[0, 0].imshow(arbitrary_frame, cmap="gray")
ax[0, 0].set_title("$G_{i,j}$")
ax[0, 1].imshow(P_hat - P_hat_center, vmin=-radiometric_range/2, vmax=radiometric_range, cmap="gray")
ax[0, 1].set_title("$\\frac{G_{i,j}}{a_{i,j}} (=P(\lambda_c, T))$")
ax[1, 0].hist(arbitrary_frame.flatten())
ax[1, 0].set_title("$G_{i,j}$ Hist " + f"(std={arbitrary_frame.std():.2e})")
ax[1, 1].hist(P_hat.flatten())
ax[1, 1].set_title("$\\frac{G_{i,j}}{a_{i,j}}" + f"(=P(\lambda_c, T))$ Hist (std={P_hat.std():.2e})")

fig.tight_layout()